#                                                                                 BERT

BERT, or Bidirectional Encoder Representations from Transformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks.

Academic paper which describes BERT in detail and provides full results on a number of tasks can be found here: https://arxiv.org/abs/1810.04805.

Github account for the paper can be found here: https://github.com/google-research/bert

BERT is a method of pre-training language representations. It is a model that has been pre-trained on a very large dataset. The unsupervised, deeply bidirectional system for pre-training NLP makes it possible for this encoder to be transfered to various downstream NLP tasks. Like our news classification problem..

The notebook can simply be run on Google Colab due to heavy (10 - 16GB) GPU requirements. TPU acess is also available through Colab.

![](https://www.lyrn.ai/wp-content/uploads/2018/11/transformer.png)

In [0]:
# If you are on Colab you can run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
%matplotlib inline
import sys
import datetime

In [0]:
#downloading weights and cofiguration file for the model
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2019-07-06 12:59:09--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.3’

uncased_L-12_H-768_ 100%[===================>] 388.84M   217MB/s    in 1.8s    

2019-07-06 12:59:11 (217 MB/s) - ‘uncased_L-12_H-768_A-12.zip.3’ saved [407727028/407727028]



In [0]:
repo = 'model_repo'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(repo)

In [0]:
# contents of the repo;
# config file for model configurations
# checkpoint (pretraining information)
!ls 'model_repo/uncased_L-12_H-768_A-12'

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2019-07-06 12:59:18--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py.3’

modeling.py.3       100%[===================>]  37.03K  --.-KB/s    in 0.01s   

2019-07-06 12:59:18 (3.65 MB/s) - ‘modeling.py.3’ saved [37922/37922]

--2019-07-06 12:59:20--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimi

Example below is done on preprocessing code, similar to **CoLa**:

The Corpus of Linguistic Acceptability is
a binary single-sentence classification task, where 
the goal is to predict whether an English sentence
is linguistically “acceptable” or not

You can use pretrained BERT model for wide variety of tasks, including classification.
The task of CoLa is close to the task of Quora competition, so I thought it woud be interesting to use that example.
Obviously, outside sources aren't allowed in Quora competition, so you won't be able to use BERT to submit a prediction.





In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
#We will use the most basic of all of them
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{repo}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{repo}/outputs'
print(f'***** Model output directory: {OUTPUT_DIR} *****')
print(f'***** BERT pretrained directory: {BERT_PRETRAINED_DIR} *****')


***** Model output directory: model_repo/outputs *****
***** BERT pretrained directory: model_repo/uncased_L-12_H-768_A-12 *****


In [0]:
from sklearn.model_selection import train_test_split
import json
# takes equal amounts of samples from each category
# returns train test data set
def create_sample(sample_amount):
    if sample_amount > 4000:
        raise ValueError('cannot sample more then 4000 data points.')
    data = pd.read_csv('drive/My Drive/bert_rte/data/wrangled_train.csv', index_col=0)
    sampled_text = []
    sampled_id = []
    sampled_tag = []
    for tag in list(data.Tag.unique()):
        sampled = data[data['Tag'] == tag].sample(sample_amount, random_state=42)
        sampled_text_ = list(sampled['Text'])
        sampled_id_ = list(sampled['ID'])
        sampled_tag_ = list(sampled['Tag_num'])

        for item in sampled_text_:
            sampled_text.append(item)
        for item in sampled_id_:
            sampled_id.append(item)
        for item in sampled_tag_:
            sampled_tag.append(item)

    with open('sampled_data.txt', 'w') as outfile:
        for sample_index in range(len(sampled_id)):
            json.dump({sampled_id[sample_index]: [sampled_text[sample_index], sampled_tag[sample_index]]}, outfile)
            outfile.write('\n')

    with open('sampled_data.txt', 'r') as infile:
        data = [json.loads(i) for i in infile if type(list(json.loads(i).values())[0][0]) == str]
    X_train, X_test, y_train, y_test = train_test_split([list(i.values())[0][0] for i in data],
                                                        [list(i.values())[0][1] for i in data], test_size=0.20,
                                                        random_state=42)

    return X_train, X_test, y_train, y_test


train_lines, test_lines, train_labels, test_labels = create_sample(2000)

In [0]:
train_labels = [i-1 for i in train_labels]
test_labels  = [i-1 for i in test_labels]

In [0]:
import modeling
import optimization
import run_classifier
import tokenization
import tensorflow as tf

# This is the ingestion method, where data is prepared for Bert 

def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 128
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(i) for i in list(set(train_labels))]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(train_lines, 'train', labels=train_labels)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

W0706 12:59:29.457189 140026542221184 deprecation_wrapper.py:119] From /content/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0706 12:59:29.461516 140026542221184 deprecation_wrapper.py:119] From /content/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0706 12:59:30.285027 140026542221184 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0706 12:59:30.287150 140026542221184 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5a1ecbba60>) includes params argument, but params are not passed to Estimat

In [0]:
"""
Note: You might see a message 'Running train on CPU'. 
This really just means that it's running on something other than a Cloud TPU, which includes a GPU.
"""

# Train the model.
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('***** Started training at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))

W0706 12:59:30.311134 140026542221184 deprecation_wrapper.py:119] From /content/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...
***** Started training at 2019-07-06 13:01:17.732805 *****
  Num examples = 36780
  Batch size = 32
***** Finished training at 2019-07-06 13:01:17.761552 *****


In [0]:
"""
There is a weird bug in original code.
When predicting, estimator returns an empty dict {}, without batch_size.
I redefine input_fn_builder and hardcode batch_size, irnoring 'params' for now.
"""

def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 32

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_examples = create_examples(test_lines, 'test', test_labels)

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = list(estimator.predict(input_fn=predict_input_fn))
predicted_classes = [np.argmax(p['probabilities']) for p in result]

{}


W0706 13:01:49.493064 140026542221184 deprecation_wrapper.py:119] From /content/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0706 13:01:49.495870 140026542221184 deprecation_wrapper.py:119] From /content/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0706 13:01:49.537412 140026542221184 deprecation_wrapper.py:119] From /content/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0706 13:01:49.596309 140026542221184 deprecation.py:323] From /content/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0706 13:01:52.124088 140026542221184 deprecation_wrapper.py:119] From /content/run_classifier.py:647: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_vari

In [0]:
from sklearn.metrics import accuracy_score

print(accuracy_score(predicted_classes, test_labels))

0.7932796868203567


There are several downsides for BERT at this moment:

- Training is expensive. All results on the paper were fine-tuned on a single Cloud TPU, which has 64GB of RAM. It is currently not possible to re-produce most of the BERT-Large results on the paper using a GPU with 12GB - 16GB of RAM, because the maximum batch size that can fit in memory is too small. 



In [0]:
predicted_classes = [i+1 for i in predicted_classes]
np.save('bert_predictions', predicted_classes)

In [0]:
np.save('bert_predictions', predicted_classes)

test_labels = [i+1 for i in test_labels]
np.save('ground_truth_10', test_labels)